In [1]:
import torch
import numpy as np
from sklearn import datasets
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import multivariate_normal

In [3]:
torch.manual_seed(0)
n_samples = 50
X = torch.rand(n_samples, 1)*2-1
Y = 0.5 * X[:, 0] - 0.3 + 0.1 * torch.randn(n_samples)

In [4]:
class BayesianLinearRegression:
    def __init__(self, alpha=1, beta=1):
        self.alpha = alpha
        self.beta = beta
    
    def fit(self, X, y):
        XtX = torch.matmul(X.T, X)
        XtX = self.beta * XtX
        XtX[np.diag_indices_from(XtX)] += self.alpha
        self.precision = XtX
        self.L_ = torch.linalg.cholesky(self.precision)
        Xy = self.beta * torch.matmul(X.T, y)
        self.w = torch.cholesky_solve(Xy.unsqueeze(1), self.L_)
        
    def predict(self, X):
        y_mean = torch.matmul(X, self.w).squeeze()
        v = torch.triangular_solve(X.T, self.L_, upper=False).solution
        y_var = 1 / self.beta + (v.T ** 2).sum(dim=1)
        return y_mean, y_var
    
    def log_likelihood(self, w, X, y):
        diff = y - torch.matmul(X, w).squeeze()
        return -0.5 * self.beta * (diff**2).sum() - 0.5 * self.alpha * (w**2).sum()

In [5]:
clf = BayesianLinearRegression(alpha=2, beta=100)
X = torch.cat([X, torch.ones(n_samples, 1)], axis=1)
clf.fit(X, Y)

In [6]:
X_test = torch.linspace(-1, 1, 100).unsqueeze(1)
X_test = torch.cat([X_test, torch.ones(100, 1)], axis=1)

In [7]:
mean, var = clf.predict(X_test)